In [5]:
import pickle
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from collections import Counter

In [2]:
with open('../data/preprocessed_X_visit_over3.pkl', 'rb') as f:
    data_x = pickle.load(f)

with open('../data/preprocessed_y_visit_over3.pkl', 'rb') as f: 
    labels = pickle.load(f)

In [ ]:
with open('../data/top_100_list.pkl', 'rb') as f:
    top_100_list = pickle.load(f)

In [4]:
visit_gt2t_new = {}
for k, v in data_x.items():
    # print(k, v)
    visit_new = {}
    for v_k, v_v in v.items():
        total_list = list(set(v_v['diagnoses'])) + list(set(v_v['procedures'])) + v_v['drugs']
        # visit_new[v_k] = v_v
        if len(total_list) > 1:
            visit_new[v_k] = {'diagnoses': None, 'procedures':None, 'drugs': None, 'admitdate':None}
            visit_new[v_k]['diagnoses'] = list(set(v_v['diagnoses'])) 
            visit_new[v_k]['procedures'] = list(set(v_v['procedures'])) 
            visit_new[v_k]['drugs'] = list(set(v_v['drugs'])) 
            visit_new[v_k]['admitdate'] = v_v['admitdate']
    visit_gt2t_new[k] = visit_new

In [ ]:
adm = pd.read_csv('../data/admissions.csv.gz', compression='gzip', usecols=['subject_id', 'hadm_id', 'admittime', 'admission_type', 'race'])
patients = pd.read_csv('../data/patients.csv')
patients = patients[['subject_id', 'gender','anchor_age','anchor_year']]
patients['yob']= patients['anchor_year'] - patients['anchor_age']

In [6]:
visit_gt2t_last = {}
for k, v in visit_gt2t_new.items():
    # break
    if len(v) >= 2:
        visit_gt2t_last[k] = v

In [7]:
## drug selection
dr_list = list()
dr_length = list()
for k, visit in visit_gt2t_new.items():
    for v_id, v in visit.items():
        dr_list.extend(v['drugs'])
        dr_length.append(len(v['drugs']))

In [8]:
max(dr_length)

276

In [9]:
drug_dict = dict(Counter(dr_list))
sorted_drug_dict = dict(sorted(drug_dict.items(), key=lambda item: item[1], reverse=False))

In [101]:
data_dict = {}
rm_patient_num =0
rm_patient_list = []

for p_id, v_data in tqdm(visit_gt2t_last.items()):
    visit_data = {'seq': [], 'data':[], 'time': [], 'diagnoses_top100': [], 'diagnoses_top100_len':[],
                  'gender': patients.loc[patients['subject_id'] == p_id, 'gender'].unique()[0],
                  'race': adm.loc[adm['subject_id'] == p_id, 'race'].unique()[0]}
    seg_idx = 0
    for v_id, record in v_data.items():
        record['visit_id'] = v_id
        record['admission_type'] = adm.loc[(adm['subject_id'] == p_id) & (adm['hadm_id'] == v_id), 'admission_type'].values[0]
        dt_obj = datetime.strptime(record['admitdate'], '%Y-%m-%d')
        age_at_adm = dt_obj.year - patients.loc[patients['subject_id'] == p_id, 'yob'].values[0]
        if age_at_adm < 0:
            age_at_adm = 90
        record['age'] = age_at_adm
        if len(record['drugs']) > 50:
            ## 약물 코드가 50개 이상이면 빈도수 기준 50개만 선택
            sorted_drugs = sorted(sorted_drug_dict, key=sorted_drug_dict.get, reverse=True)
            top100_drugs = [elem for elem in sorted_drugs if elem in record['drugs']][:50]
            seq_data = record['diagnoses'] + record['procedures'] + top100_drugs
        else:
            seq_data = record['diagnoses'] + record['procedures'] + record['drugs']
        
        top100_diagnoses = [d_code for d_code in record['diagnoses'] if d_code in top_100_list]
        visit_data['diagnoses_top100'].append(top100_diagnoses)
        visit_data['diagnoses_top100_len'].append(len(top100_diagnoses))
        visit_data['seq'].append(seq_data)
        visit_data['data'].append(record)
        visit_data['time'].append(dt_obj)
        
    if 0 in visit_data['diagnoses_top100_len']:
        rm_patient_num += 1
        rm_patient_list.append(p_id)
        continue
    
    td_list = [(visit_data['time'][-1]- dt).days for dt in visit_data['time']]
    visit_data['timedelta'] = td_list
    visit_data['labels_origin'] = labels[p_id]['top100_label']
    visit_data['labels_bin'] = labels[p_id]['top100_label_bin']
    data_dict[p_id] = visit_data

100%|██████████| 43096/43096 [05:54<00:00, 121.47it/s]


In [102]:
len(data_dict)

39802

In [103]:
# with open('../new_data/data4pretrained_filter_drugs.pkl', 'wb') as f:
#     pickle.dump(data_dict, f)

In [104]:
with open('../new_data/data4pretrained_filter_drugs.pkl', 'rb') as f:
    data_dict = pickle.load(f)

In [105]:
d_list = list()
p_list = list()
dr_list = list()
for k, visit in data_dict.items():
    for v in visit['data']:
        d_list.extend(v['diagnoses'])
        p_list.extend(v['procedures'])
        dr_list.extend(v['drugs'])
        
d_list = list(set(d_list))
p_list = list(set(p_list))
dr_list = list(set(dr_list))

In [106]:
len(d_list), len(p_list), len(dr_list)

(863, 738, 3487)

In [107]:
code2idx = {'padding':0}

idx = 1
for d in d_list:
    code2idx[d] = idx
    idx += 1

for p in p_list:
    code2idx[p] = idx
    idx += 1

dr_idx = 1
for dr in dr_list:  
    code2idx[dr] = idx
    idx += 1

code2idx['[CLS]'] = idx
code2idx['[SEP]'] = idx + 1
code2idx['[MASK]'] = idx + 2

In [108]:
len(code2idx)

5092

In [109]:
code2idx['[MASK]']

5091

In [110]:
# with open('../new_data/code_indices/code_dict_pretrained.pkl', 'wb') as f:
#     pickle.dump(code2idx, f)

In [50]:
with open('../new_data/code_indices/code_dict_pretrained.pkl', 'rb') as f:
    code2idx = pickle.load(f)

In [112]:
data_dict_idx = data_dict.copy()

In [113]:
for k, visit in tqdm(data_dict_idx.items()):
    visit['seq_idx'] = [list(map(code2idx.get, v)) for v in visit['seq']]

100%|██████████| 39802/39802 [00:00<00:00, 62178.13it/s]


In [114]:
# for k, visit in tqdm(data_dict_idx.items()):
#     visit['total_seq_idx'] = list(map(code2idx.get, visit['total_seq']))

In [119]:
data_dict_new= dict()
rm_patient_num =0
rm_patient_list = []
for p_id in tqdm(list(data_dict_idx.keys())):
    p_data = {'visit':[], 'visit_idx': [], 'visit_length':0, 'age': [], 'gender': None, 'race': None, 'admission_type': [], \
              'diagnoses_top100': None, 'diagnoses_top100_len': None, 'seq':None, 'seq_idx':None, 'timedelta': None, 'label':[]}
    
    for v_id, v_data in enumerate(data_dict_idx[p_id]['data']):            
        p_data['visit'].append(v_data['visit_id'])
        p_data['visit_idx'].append(v_id)
        p_data['seq'] = data_dict_idx[p_id]['seq']
        p_data['seq_idx'] = data_dict_idx[p_id]['seq_idx']
        p_data['time'] = data_dict_idx[p_id]['time']
        p_data['timedelta'] = data_dict_idx[p_id]['timedelta']
        p_data['age'].append(v_data['age'])
        p_data['admission_type'].append(v_data['admission_type'])
    
    p_data['diagnoses_top100'] = data_dict_idx[p_id]['diagnoses_top100']
    p_data['diagnoses_top100_len'] = data_dict_idx[p_id]['diagnoses_top100_len']
    assert len(p_data['diagnoses_top100']) == len(p_data['diagnoses_top100_len']) == len(p_data['visit'])
    p_data['race'] = data_dict_idx[p_id]['race']
    p_data['gender'] = data_dict_idx[p_id]['gender']
    p_data['label'] = np.squeeze(data_dict_idx[p_id]['labels_bin'])
    data_dict_new[p_id] = p_data

100%|██████████| 39802/39802 [00:00<00:00, 91621.19it/s]


In [123]:
# with open('../new_data/data4pretrained_filter_drugs_preprocess.pkl', 'wb') as f:
#     pickle.dump(data_dict_new, f)

In [18]:
with open('../new_data/data4pretrained_filter_drugs_preprocess.pkl', 'rb') as f:
    data_dict_new = pickle.load(f)

In [126]:
len(data_dict_new)

39802

In [35]:
data_dict_max50 = dict()
for p_id , data in data_dict_new.items():
    new_data = dict()
    if len(data['visit']) > 50:
        print(p_id, len(data['visit']))
        new_visit_idx = [i for i in range(len(data['visit_idx'][-50:]))]
    else:
        new_visit_idx = data['visit_idx']
    new_data['visit'] = data['visit'][-50:]
    new_data['visit_idx'] = new_visit_idx
    new_data['visit_length'] = len(data['visit'][-50:])
    new_data['seq'] = data['seq'][-50:]
    new_data['seq_idx'] = data['seq_idx'][-50:]
    new_data['time'] = data['time'][-50:]
    new_data['timedelta'] = data['timedelta'][-50:]
    new_data['age'] = data['age'][-50:]
    new_data['diagnoses_top100'] = data['diagnoses_top100'][-50:]
    new_data['diagnoses_top100_len'] = data['diagnoses_top100_len'][-50:]
    assert len(data['visit_idx'][-50:]) == len(data['seq'][-50:]) == len(data['age'][-50:]) == len(data['diagnoses_top100'][-50:])
    new_data['gender'] = data['gender']
    new_data['race'] = data['race']
    new_data['label'] = data['label']
    data_dict_max50[p_id] = new_data

10123949 55
10264646 76
10577647 78
10578325 67
10580201 59
10714009 113
10913302 64
11021643 63
11296936 76
11553072 69
11582633 81
11714071 62
11761621 56
11818101 64
11890447 66
12251785 79
12468016 88
12547294 69
12563258 60
12596559 63
13166511 53
13297743 91
13470788 52
13475033 96
13877234 60
13999829 54
14029474 52
14394983 124
15084163 53
15107347 51
15108590 52
15114531 67
15229574 79
15464144 75
15496609 133
15935768 56
16124481 62
16233333 67
16439884 58
16615356 55
16662316 129
16675371 66
16809525 57
16924675 54
17011846 56
17051420 54
17204468 59
17340686 51
17477304 51
17517983 94
17716210 80
17937834 61
18001923 69
18284271 82
18376342 52
18553055 55
18655830 69
18676703 59
18970086 68
19127408 51
19610016 52
19713100 62
19759225 63
19921471 57


In [36]:
# with open('../new_data/data4pretrained_filter_drugs_preprocess_maxlen50.pkl', 'wb') as f:
#     pickle.dump(data_dict_max50, f)

In [41]:
with open('../new_data/data4pretrained_filter_drugs_preprocess_maxlen50.pkl', 'rb') as f:
    data_dict_max50 = pickle.load(f)

In [42]:
def convert_code_type(code):
    if code.startswith('d'):
        return 1
    elif code.startswith('pcs'):
        return 2
    elif code.startswith('p'):
        return 3
    elif code in ['[CLS]', '[SEP]']:
        return 4

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

In [44]:
mlb = MultiLabelBinarizer()
mlb.fit([top_100_list])

MultiLabelBinarizer()

In [45]:
mlb.classes_

array(['d_008', 'd_038', 'd_041', 'd_070', 'd_197', 'd_198', 'd_244',
       'd_250', 'd_263', 'd_272', 'd_274', 'd_275', 'd_276', 'd_278',
       'd_280', 'd_284', 'd_285', 'd_287', 'd_288', 'd_294', 'd_295',
       'd_296', 'd_300', 'd_303', 'd_305', 'd_309', 'd_311', 'd_327',
       'd_338', 'd_345', 'd_346', 'd_348', 'd_357', 'd_362', 'd_365',
       'd_401', 'd_403', 'd_404', 'd_410', 'd_412', 'd_414', 'd_416',
       'd_424', 'd_425', 'd_426', 'd_427', 'd_428', 'd_438', 'd_440',
       'd_443', 'd_453', 'd_456', 'd_458', 'd_459', 'd_486', 'd_491',
       'd_493', 'd_496', 'd_511', 'd_518', 'd_530', 'd_536', 'd_553',
       'd_560', 'd_562', 'd_564', 'd_569', 'd_571', 'd_572', 'd_576',
       'd_577', 'd_578', 'd_584', 'd_585', 'd_599', 'd_600', 'd_648',
       'd_682', 'd_707', 'd_715', 'd_719', 'd_724', 'd_729', 'd_733',
       'd_780', 'd_781', 'd_782', 'd_783', 'd_784', 'd_785', 'd_786',
       'd_787', 'd_788', 'd_789', 'd_790', 'd_799', 'd_995', 'd_996',
       'd_997', 'd_9

In [46]:
seed_list = [123, 321, 666, 777, 5959]

In [47]:
total_patient_list = list(data_dict_max50.keys())

In [144]:
# for seed in seed_list:
#     train_indices, test_indices = train_test_split(total_patient_list, test_size=0.1, random_state=seed)
#     train_indices, val_indices = train_test_split(train_indices, test_size=len(test_indices), random_state=seed)
#     print(len(train_indices), len(val_indices), len(test_indices))
#     with open(f'../new_data/split_indices/train_indices_{seed}.pkl', 'wb') as f:
#         pickle.dump(train_indices, f)
#     with open(f'../new_data/split_indices/val_indices_{seed}.pkl', 'wb') as f:
#         pickle.dump(val_indices, f)
#     with open(f'../new_data/split_indices/test_indices_{seed}.pkl', 'wb') as f:
#         pickle.dump(test_indices, f)

31840 3981 3981
31840 3981 3981
31840 3981 3981
31840 3981 3981
31840 3981 3981


In [48]:
slide_length = 5

In [51]:
for seed in seed_list:
    with open(f'../new_data/split_indices/train_indices_{seed}.pkl', 'rb') as f:
        train_indices = pickle.load(f)
    f.close()
    
    total_len_list = []
    train_datasets = dict()
    for p_id in tqdm(train_indices):
        data = data_dict_max50[p_id]
        if len(data['visit']) > slide_length:
            for i in range(len(data['visit']) - slide_length + 1):
                visit_data = dict()
                visit_data['patient_id'] = p_id
                visit_data['visit'] = data['visit'][i:i+slide_length]
                visit_data['visit_idx'] = data['visit_idx'][i:i+slide_length]
                visit_data['visit_length'] = len(data['visit'][i:i+slide_length])
                
                slide_seq = data['seq'][i:i+slide_length]
                slide_seq_idx = data['seq_idx'][i:i+slide_length]
                
                total_seq = ['[CLS]'] + [item for sublist in slide_seq for item in sublist + ['[SEP]']]
                total_seq_idx = [code2idx['[CLS]']] + [item for sublist in slide_seq_idx for item in sublist + [code2idx['[SEP]']]]
                total_code_type = list(map(convert_code_type, total_seq))
                total_vis_seg = [visit_data['visit_idx'][0]] + [elem for idx, v_idx in enumerate(visit_data['visit_idx']) for elem in [v_idx] * len(slide_seq_idx[idx]) + [v_idx]]
                assert len(total_seq) == len(total_seq_idx) == len(total_code_type) == len(total_vis_seg)
                
                visit_data['total_seq'] = total_seq
                visit_data['total_seq_idx'] = total_seq_idx
                visit_data['code_type'] = total_code_type
                visit_data['visit_seg'] = total_vis_seg
                visit_data['total_len'] = len(total_seq)
                visit_data['time'] = data['time'][i:i+slide_length]

                timedelta_replacement = {visit_data['visit_idx'][idx]: td for idx, td in enumerate(data['timedelta'][i:i+slide_length])}
                visit_data['timedelta'] = [timedelta_replacement[val] for val in total_vis_seg]
                timevec_replacement = {visit_data['visit_idx'][idx]: [tm.year, tm.month, tm.day, tm.isocalendar().week] \
                                    for idx, tm in enumerate(data['time'][i:i+slide_length])}
                visit_data['timevec'] = [timevec_replacement[val] for val in total_vis_seg]
                if i == len(data['visit']) - slide_length :
                    visit_data['label'] = data['label']
                    visit_data['label_visit_idx'] = data['visit_idx'][-1] + 1
                else:
                    visit_data['label'] = mlb.transform([data['diagnoses_top100'][i+slide_length]])[0]
                    visit_data['label_visit_idx'] = data['visit_idx'][i+slide_length]
                train_datasets[f'{str(p_id)}_slide{str(i)}'] = visit_data
        else:
            visit_data = dict()
            visit_data['patient_id'] = p_id
            visit_data['visit'] = data['visit']
            visit_data['visit_idx'] = data['visit_idx']
            visit_data['visit_length'] = data['visit_length']
            
            total_seq = ['[CLS]'] + [item for sublist in data['seq'] for item in sublist + ['[SEP]']]
            total_seq_idx = [code2idx['[CLS]']] + [item for sublist in data['seq_idx'] for item in sublist + [code2idx['[SEP]']]]
            total_code_type = list(map(convert_code_type, total_seq))
            total_vis_seg = [data['visit_idx'][0]] + [elem for idx, v_idx in enumerate(data['visit_idx']) for elem in [v_idx] * len(data['seq_idx'][idx]) + [v_idx]]
            assert len(total_seq) == len(total_seq_idx) == len(total_code_type) == len(total_vis_seg)
            
            visit_data['total_seq'] = total_seq
            visit_data['total_seq_idx'] = total_seq_idx
            visit_data['code_type'] = total_code_type
            visit_data['visit_seg'] = total_vis_seg
            visit_data['total_len'] = len(total_seq)
            visit_data['time'] = data['time']
            
            timedelta_replacement = {visit_data['visit_idx'][idx]: td for idx, td in enumerate(data['timedelta'])}
            visit_data['timedelta'] = [timedelta_replacement[val] for val in total_vis_seg]
            timevec_replacement = {visit_data['visit_idx'][idx]: [tm.year, tm.month, tm.day, tm.isocalendar().week] \
                                for idx, tm in enumerate(data['time'])}
            visit_data['timevec'] = [timevec_replacement[val] for val in total_vis_seg]
            visit_data['label'] = data['label']
            visit_data['label_visit_idx'] = data['visit_idx'][-1] + 1
            train_datasets[f'{str(p_id)}_slide{str(0)}'] = visit_data
            
        total_len_list.append(len(total_seq))
    
    print(max(total_len_list))
    with open(f'../new_data/split_datasets/train_datasets_{seed}.pkl', 'wb') as f:
        pickle.dump(train_datasets, f)

  0%|          | 0/31840 [00:00<?, ?it/s]

100%|██████████| 31840/31840 [00:13<00:00, 2425.46it/s]


395


100%|██████████| 31840/31840 [00:14<00:00, 2124.79it/s]


395


...

395


100%|██████████| 31840/31840 [00:14<00:00, 2151.13it/s]


381


100%|██████████| 31840/31840 [00:12<00:00, 2475.55it/s]


395


In [52]:
for seed in seed_list:
    with open(f'../new_data/split_indices/val_indices_{seed}.pkl', 'rb') as f:
        val_indices = pickle.load(f)
    f.close()
    
    total_len_list = []
    valid_datasets = dict()
    for p_id in tqdm(val_indices):
        data = data_dict_max50[p_id]
        visit_data = dict()
        visit_data['patient_id'] = p_id
        visit_data['visit'] = data['visit'][-slide_length:]
        visit_data['visit_idx'] = data['visit_idx'][-slide_length:]
        visit_data['visit_length'] = len(data['visit'][-slide_length:])

        slide_seq = data['seq'][-slide_length:]
        slide_seq_idx = data['seq_idx'][-slide_length:]

        total_seq = ['[CLS]'] + [item for sublist in slide_seq for item in sublist + ['[SEP]']]
        total_seq_idx = [code2idx['[CLS]']] + [item for sublist in slide_seq_idx for item in sublist + [code2idx['[SEP]']]]
        total_code_type = list(map(convert_code_type, total_seq))
        total_vis_seg = [visit_data['visit_idx'][0]] + [elem for idx, v_idx in enumerate(visit_data['visit_idx']) for elem in [v_idx] * len(slide_seq_idx[idx]) + [v_idx]]
        assert len(total_seq) == len(total_seq_idx) == len(total_code_type) == len(total_vis_seg)

        visit_data['total_seq'] = total_seq
        visit_data['total_seq_idx'] = total_seq_idx
        visit_data['code_type'] = total_code_type
        visit_data['visit_seg'] = total_vis_seg
        visit_data['total_len'] = len(total_seq)
        visit_data['time'] = data['time'][-slide_length:]

        timedelta_replacement = {visit_data['visit_idx'][idx]: td for idx, td in enumerate(data['timedelta'][-slide_length:])}
        visit_data['timedelta'] = [timedelta_replacement[val] for val in total_vis_seg]
        timevec_replacement = {visit_data['visit_idx'][idx]: [tm.year, tm.month, tm.day, tm.isocalendar().week] \
                            for idx, tm in enumerate(data['time'][-slide_length:])}
        visit_data['timevec'] = [timevec_replacement[val] for val in total_vis_seg]
        visit_data['label'] = data['label']
        visit_data['label_visit_idx'] = data['visit_idx'][-1] + 1
        valid_datasets[f'{str(p_id)}_slide{str(0)}'] = visit_data
        total_len_list.append(len(total_seq))
    print(max(total_len_list))
    with open(f'../new_data/split_datasets/valid_datasets_{seed}.pkl', 'wb') as f:
        pickle.dump(valid_datasets, f)

100%|██████████| 3981/3981 [00:00<00:00, 11543.03it/s]


367


100%|██████████| 3981/3981 [00:00<00:00, 11619.22it/s]


386


100%|██████████| 3981/3981 [00:00<00:00, 11691.95it/s]


371


100%|██████████| 3981/3981 [00:00<00:00, 11530.81it/s]


395


100%|██████████| 3981/3981 [00:00<00:00, 11542.27it/s]


384


In [53]:
for seed in seed_list:
    with open(f'../new_data/split_indices/test_indices_{seed}.pkl', 'rb') as f:
        test_indices = pickle.load(f)
    f.close()
    
    total_len_list = []
    test_datasets = dict()
    for p_id in tqdm(test_indices):
        data = data_dict_max50[p_id]
        visit_data = dict()
        visit_data['patient_id'] = p_id
        visit_data['visit'] = data['visit'][-slide_length:]
        visit_data['visit_idx'] = data['visit_idx'][-slide_length:]
        visit_data['visit_length'] = len(data['visit'][-slide_length:])

        slide_seq = data['seq'][-slide_length:]
        slide_seq_idx = data['seq_idx'][-slide_length:]

        total_seq = ['[CLS]'] + [item for sublist in slide_seq for item in sublist + ['[SEP]']]
        total_seq_idx = [code2idx['[CLS]']] + [item for sublist in slide_seq_idx for item in sublist + [code2idx['[SEP]']]]
        total_code_type = list(map(convert_code_type, total_seq))
        total_vis_seg = [visit_data['visit_idx'][0]] + [elem for idx, v_idx in enumerate(visit_data['visit_idx']) for elem in [v_idx] * len(slide_seq_idx[idx]) + [v_idx]]
        assert len(total_seq) == len(total_seq_idx) == len(total_code_type) == len(total_vis_seg)

        visit_data['total_seq'] = total_seq
        visit_data['total_seq_idx'] = total_seq_idx
        visit_data['code_type'] = total_code_type
        visit_data['visit_seg'] = total_vis_seg
        visit_data['total_len'] = len(total_seq)
        visit_data['time'] = data['time'][-slide_length:]

        timedelta_replacement = {visit_data['visit_idx'][idx]: td for idx, td in enumerate(data['timedelta'][-slide_length:])}
        visit_data['timedelta'] = [timedelta_replacement[val] for val in total_vis_seg]
        timevec_replacement = {visit_data['visit_idx'][idx]: [tm.year, tm.month, tm.day, tm.isocalendar().week] \
                            for idx, tm in enumerate(data['time'][-slide_length:])}
        visit_data['timevec'] = [timevec_replacement[val] for val in total_vis_seg]
        visit_data['label'] = data['label']
        visit_data['label_visit_idx'] = data['visit_idx'][-1] + 1
        test_datasets[f'{str(p_id)}_slide{str(0)}'] = visit_data
        total_len_list.append(len(total_seq))
        
    print(max(total_len_list))
    with open(f'../new_data/split_datasets/test_datasets_{seed}.pkl', 'wb') as f:
        pickle.dump(test_datasets, f)

100%|██████████| 3981/3981 [00:00<00:00, 11624.44it/s]


380


100%|██████████| 3981/3981 [00:00<00:00, 11465.05it/s]


380


100%|██████████| 3981/3981 [00:00<00:00, 11656.49it/s]


365


100%|██████████| 3981/3981 [00:00<00:00, 11639.84it/s]


391


100%|██████████| 3981/3981 [00:00<00:00, 11634.97it/s]


381


In [197]:
min(total_len_list)

7